# Shark Attacks

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime as dt

In [2]:
shark_atck_df = pd.read_csv('attacks.csv', encoding='latin-1')
shark_atck_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
shark_atck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

The dataframe has a total of 25723 entries and 24 columns. All of the columns have more than 17,000 null values. Checking all the columns' names, I can deduce that not all of them will be useful for my research. Let's select the ones that will be, and generate a new dataframe with them:

 - Case Number, Date, Year, Case Number 1 & 2: checking the sample above I can see that every of those columns is related with the date the attack took place. The one with more non-null values is 'Case Number', with 8702. I would like to check if this column covers the rest (if some dates are not in 'Case Number' but in other column).

In [4]:
#dates = shark_atck_df.iloc[:,[0,1,2,19,20]]
#sns.heatmap(dates.isnull(), cbar=False)

As the heatmap shows, all the dates in the rest of the columns and beyond are covered by 'Case Number', so I will use that column as date.

 - Time: with nearly 3000 non-nulls, I will keep it as it could be usefull.
 - Country, Area, Location: more than 5700 rows with all this information covered. The 'Country' column will be the main source related with this information, but I prefer to keep also 'Area' and 'Location', even though they have less non-null values.
 - Sex, Age: more than 5700 non-null values for sex; more than 3400 for Age.
 - Activity: what the person attacked was doing when the attack happened. There are more than 5700 non-nulls.
 - Type: the type of attack, there are nearly 6300 non-null values.
 - Species: the species of the shark, with nearly 3500 non-null values.
 - Injury & Fatal: the kind of injury and if it was rather fatal or not. Nearly 6300 non-null entries for the injuries; more than 5700 for the 'Fatal' column.

Let's create the new dataframe with the selected columns sorted, and change the name of 'Case Number' to 'Date'.

In [5]:
attacks_df = shark_atck_df.iloc[:,[0,13,4,5,6,9,10,7,3,14,11,12]]
attacks_df.rename(columns={'Case Number': 'Date'}, inplace=True)
attacks_df.head()

,Date,Time,Country,Area,Location,Sex,Age,Activity,Type,Species,Injury,Fatal (Y/N)
0,2018.06.25,18h00,USA,California,"Oceanside, San Diego County",F,57,Paddling,Boating,White shark,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",F,11,Standing,Unprovoked,NaN,Minor injury to left thigh,N
2,2018.06.09,07h45,USA,Hawaii,"Habush, Oahu",M,48,Surfing,Invalid,NaN,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,NaN,AUSTRALIA,New South Wales,Arrawarra Headland,M,NaN,Surfing,Unprovoked,2 m shark,Minor injury to lower leg,N
4,2018.06.04,NaN,MEXICO,Colima,La Ticla,M,NaN,Free diving,Provoked,"Tiger shark, 3m",Lacerations to leg & hand shark PROVOKED INCIDENT,N


Now let's see through a heatmap how is the distribution of the non-null values in our new dataframe, in order to see how many rows have all its values as null:

In [6]:
#sns.heatmap(attacks_df.isnull(), cbar=False)

In [7]:
attacks_df.count().sort_values(ascending=False)

Date           8702
Type           6298
Injury         6274
Country        6252
Area           5847
Fatal (Y/N)    5763
Location       5762
Activity       5758
Sex            5737
Age            3471
Species        3464
Time           2948
dtype: int64

As the heatmap shows, 2/3 of the dataframe has all its rows filled with null values in absolutely all its columns. Of course, all this rows will be deleted. Of the reminder third, at least 1/4 has just the 'Date' column filled, so I will also dump all those values and create a separate series with all the dates, just in case I want to perform a historical of shark attacks throught the years or something similar. 

In order to attemp the dataframe, I will keep those rows where the values in the column 'Type' are non-null, as it is the column with the largest number of those after 'Date'. This will let me with a 6298-row dataframe.

In [8]:
dates_s = attacks_df['Date'].dropna() #Series with all the dates
attacks_df.dropna(how='all', subset=['Type'], inplace=True) #Dataframe
attacks_df

,Date,Time,Country,Area,Location,Sex,Age,Activity,Type,Species,Injury,Fatal (Y/N)
0,2018.06.25,18h00,USA,California,"Oceanside, San Diego County",F,57,Paddling,Boating,White shark,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",F,11,Standing,Unprovoked,NaN,Minor injury to left thigh,N
2,2018.06.09,07h45,USA,Hawaii,"Habush, Oahu",M,48,Surfing,Invalid,NaN,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,NaN,AUSTRALIA,New South Wales,Arrawarra Headland,M,NaN,Surfing,Unprovoked,2 m shark,Minor injury to lower leg,N
4,2018.06.04,NaN,MEXICO,Colima,La Ticla,M,NaN,Free diving,Provoked,"Tiger shark, 3m",Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,NaN,AUSTRALIA,Western Australia,Roebuck Bay,M,NaN,Diving,Unprovoked,NaN,FATAL,Y
6298,ND.0004,NaN,AUSTRALIA,Western Australia,NaN,M,NaN,Pearl diving,Unprovoked,NaN,FATAL,Y
6299,ND.0003,NaN,USA,North Carolina,Ocracoke Inlet,M,NaN,Swimming,Unprovoked,NaN,FATAL,Y
6300,ND.0002,NaN,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",M,NaN,NaN,Unprovoked,NaN,FATAL,Y


Checking column by column, the first possible problem we can see is the format of the 'Date' column. As it can be seen with the value_counts() function, some dates have letters, marks or other characters like '.', '/' or similar. I should remove every character that is not numerical from the date, so afterwards I can get year, month and day from the reminder values. 

As it is shown above, the dates are collected, in general, with the format 'YYYY.MM.DD' as a string. So, what I can do is slice that string with indexes and then assign those to the column 'Year', 'Month' or 'Day'. Let's perform the operation.

In [9]:
attacks_df['Date'] = [re.sub(r"[A-Za-z.,&-/ ]", "", str(x)) for x in attacks_df['Date']]
attacks_df['Date']

0       20180625
1       20180618
2       20180609
3       20180608
4       20180604
          ...   
6297        0005
6298        0004
6299        0003
6300        0002
6301        0001
Name: Date, Length: 6298, dtype: object

Checking the reminder strings, we can see there are some cases where the expected format is not fulfilled (strngs are either bigger or smaller). Those are just a few cases, so checking them more in detail, we can see that, in cases where the lenght of the string is larger, there is just one more number at the end that should be removed; whilst in cases where the string is shorter, there is just one date that probably is missing its day, while the others seem to be cases named with a different nomenclature.

In [10]:
fechas_m = [x for x in attacks_df['Date'] if len(x) > 8]
fechas_l = [x for x in attacks_df['Date'] if len(x) < 8]
print(fechas_m)
print(fechas_l)

['200206132', '200206131', '186300004', '186300003', '186300002']
['', '186405', '0154', '0153', '0152', '0151', '0150', '0149', '0148', '1940', '0139', '0138', '0136', '0135', '0134', '0133', '0132', '0130', '0129', '0127', '0124', '0123', '0122', '0119', '0118', '0116', '0115', '0114', '0113', '0111', '0110', '0109', '0108', '0107', '0106', '0104', '0102', '0100', '0097', '0096', '0095', '0094', '0093', '0091', '0090', '0089', '0088', '0087', '0086', '0085', '0084', '0083', '0082', '0081', '0078', '0076', '0075', '0074', '0073', '0069', '0068', '0066', '0065', '0064', '0063', '0062', '0060', '0059', '0058', '0057', '0056', '0055', '0054', '0053', '0052', '0051', '0049', '0048', '0047', '0046', '0044', '0043', '0042', '0041', '0040', '0039', '0038', '0037', '0036', '0035', '0034', '0033', '0032', '0031', '0030', '0028', '0027', '0026', '0025', '0024', '0023', '0022', '0021', '0020', '0019', '0018', '0017', '0016', '0015', '0014', '0013', '0012', '0011', '0010', '0009', '0008', '0007',

Let's try to fix the larger dates and discover more about the shorter

In [11]:
attacks_df['Date'] = [x[:8] if len(x) > 8 else x for x in attacks_df['Date']]
[x for x in attacks_df['Date'] if len(x) > 8]

[]

### Shorter date strings

In order to check on those date that does not accomplish with the standards, I will create a new dataframe with all the columns that where related with the date of the attack in the original dataframe. These are 'Case Number', 'Date', 'Year', 'Case Number.1' and 'Case Number.2'. I will also dump the null values and standarize the strings in 'Case Number', as I done in 'attacks_df', in order to get the correlative rows so I can merge data after this analysis if necessary.

In [12]:
dates = shark_atck_df.iloc[:,[0,1,2,19,20]]

In [13]:
dates.replace('nan', np.NaN, inplace=True)
dates.replace(np.nan, np.NaN, inplace=True)
dates.replace('0', np.NaN, inplace=True)
dates.replace(0.0, np.NaN, inplace=True)
dates.replace('xx', np.NaN, inplace=True)

In [14]:
dates.dropna(how='all', inplace=True)

In [15]:
dates['Case Number'] = [re.sub(r"[A-Za-z.,&-/ ]", "", str(x)) for x in dates['Case Number']]
dates

,Case Number,Date,Year,Case Number.1,Case Number.2
0,20180625,25-Jun-2018,2018.0,2018.06.25,2018.06.25
1,20180618,18-Jun-2018,2018.0,2018.06.18,2018.06.18
2,20180609,09-Jun-2018,2018.0,2018.06.09,2018.06.09
3,20180608,08-Jun-2018,2018.0,2018.06.08,2018.06.08
4,20180604,04-Jun-2018,2018.0,2018.06.04,2018.06.04
...,...,...,...,...,...
6297,0005,Before 1903,NaN,ND.0005,ND.0005
6298,0004,Before 1903,NaN,ND.0004,ND.0004
6299,0003,1900-1905,NaN,ND.0003,ND.0003
6300,0002,1883-1889,NaN,ND.0002,ND.0002


As we can see, 'dates' dataframe has the same number of rows of 'attacks_df', and the indexes correspond to the same values; so this new dataframe could be merged with no problems to 'attacks_df' in case we need it. Now, let's select all those rows where the string in the column 'Case Number' has a lenght shorter than 8 characters, and see if the rest of the columns could give us any clue about the dates of them.

In [16]:
dates_short = dates[dates['Case Number'].str.len() < 8]
dates_short.head(15)

,Case Number,Date,Year,Case Number.1,Case Number.2
5488,,Reported 06-Sep-1905,1905.0,1905.09.06.R,1905.09.06.R
5944,186405,May-1864,1864.0,1864.05.00,1864.05.00
6181,0154,Before 1939,NaN,ND-0154,ND-0154
6182,0153,1990 or 1991,NaN,ND-0153,ND-0153
6183,0152,Before 2016,NaN,ND-0152,ND-0152
6184,0151,Before Oct-2009,NaN,ND-0151,ND-0151
6185,0150,Before 1934,NaN,ND-0150,ND-0150
6186,0149,Before 1934,NaN,ND-0149,ND-0149
6187,0148,2009?,NaN,ND-0148,ND-0148
6188,1940,Before 1930,NaN,ND-1940,ND-1940


Showing a few rows that accomplish that condition, I can see there are two on top that have a more completed date; and also that the 'Date' column has the years of the attack (or the range of years where the attack possibly took place) for some other rows, but nothing else. Let's keep just the year (or the last in the ranges), append to it a '0000' to complete the date and put the values in a new column.

In [17]:
dates_short['Date'] = [re.sub(r"[A-Za-z.,&-/ ?]", "", str(x)) for x in dates_short['Date']]
dates_short = dates_short[dates_short['Date'].str.len() >= 4]
dates_short['Case Number'] = [x[-4:]+'0000' for x in dates_short['Date']]

In [18]:
dates_short.head()

,Case Number,Date,Year,Case Number.1,Case Number.2
5488,19050000,061905,1905.0,1905.09.06.R,1905.09.06.R
5944,18640000,1864,1864.0,1864.05.00,1864.05.00
6181,19390000,1939,NaN,ND-0154,ND-0154
6182,19910000,19901991,NaN,ND-0153,ND-0153
6183,20160000,2016,NaN,ND-0152,ND-0152


Now 'Case Number' column has all the years in the same format as it is in 'attacks_df'. As we know more details about the two dates on top of the dataframe, let's also change them, so we get more information about those rows.

In [19]:
dates_short['Case Number'][5488] = '19050906'
dates_short['Case Number'][5944] = '18640500'

In [20]:
dates_short.tail()

,Case Number,Date,Year,Case Number.1,Case Number.2
6297,19030000,1903,NaN,ND.0005,ND.0005
6298,19030000,1903,NaN,ND.0004,ND.0004
6299,19050000,19001905,NaN,ND.0003,ND.0003
6300,18890000,18831889,NaN,ND.0002,ND.0002
6301,18530000,18451853,NaN,ND.0001,ND.0001


In [21]:
new_date = pd.concat([attacks_df['Date'], dates_short['Case Number']], axis=1)
new_date

,Date,Case Number
0,20180625,NaN
1,20180618,NaN
2,20180609,NaN
3,20180608,NaN
4,20180604,NaN
...,...,...
6297,0005,19030000
6298,0004,19030000
6299,0003,19050000
6300,0002,18890000


In [26]:
attacks_df['Rest Dates'] = dates_short['Case Number']
attacks_df['Rest Dates'].fillna(attacks_df['Date'], inplace=True)
attacks_df['Date'] = attacks_df['Rest Dates']
attacks_df.drop(['Rest Dates'], axis=1)

,Date,Time,Country,Area,Location,Sex,Age,Activity,Type,Species,Injury,Fatal (Y/N)
0,20180625,18h00,USA,California,"Oceanside, San Diego County",F,57,Paddling,Boating,White shark,"No injury to occupant, outrigger canoe and pad...",N
1,20180618,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",F,11,Standing,Unprovoked,NaN,Minor injury to left thigh,N
2,20180609,07h45,USA,Hawaii,"Habush, Oahu",M,48,Surfing,Invalid,NaN,Injury to left lower leg from surfboard skeg,N
3,20180608,NaN,AUSTRALIA,New South Wales,Arrawarra Headland,M,NaN,Surfing,Unprovoked,2 m shark,Minor injury to lower leg,N
4,20180604,NaN,MEXICO,Colima,La Ticla,M,NaN,Free diving,Provoked,"Tiger shark, 3m",Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,19030000,NaN,AUSTRALIA,Western Australia,Roebuck Bay,M,NaN,Diving,Unprovoked,NaN,FATAL,Y
6298,19030000,NaN,AUSTRALIA,Western Australia,NaN,M,NaN,Pearl diving,Unprovoked,NaN,FATAL,Y
6299,19050000,NaN,USA,North Carolina,Ocracoke Inlet,M,NaN,Swimming,Unprovoked,NaN,FATAL,Y
6300,18890000,NaN,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",M,NaN,NaN,Unprovoked,NaN,FATAL,Y
